In [1]:
import numpy as np
import pandas as pd
import os
import time
import csv
import shutil

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model

Using TensorFlow backend.


In [2]:
PATH = "C:/Users/novanto/Downloads/AI AU Materials/FINAL PROJECT 4"
data_path = PATH + '/train_images'
data_dir_list = os.listdir(data_path)

In [3]:
img_data_list=[]

for dataset in data_dir_list:
  img_list = os.listdir(data_path + '/' + dataset)
  print('Loaded the images of dataset-'+dataset)
  for img in img_list:
    img_path = data_path +  '/' + dataset + '/' + img
    img = image.load_img(img_path)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    img_data_list.append(x)

img_data = np.array(img_data_list)

print (img_data.shape)  #(600, 1, 224, 224, 3)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)  #(1, 600, 224, 224, 3)
img_data=img_data[0]
print (img_data.shape)  #(600, 224, 224, 3)

Loaded the images of dataset-0
Loaded the images of dataset-1
Loaded the images of dataset-2
Loaded the images of dataset-3
Loaded the images of dataset-4
Loaded the images of dataset-5
(2528, 1, 224, 224, 3)
(1, 2528, 224, 224, 3)
(2528, 224, 224, 3)


In [4]:
num_classes = 6
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:100]=0
labels[100:200]=1
labels[200:300]=2
labels[300:400]=3
labels[400:500]=4
labels[500:600]=5

# one-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

x,y = shuffle(img_data,Y, random_state=2)

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=2)

In [6]:
# from keras.applications.resnet50 import ResNet50
# from keras.layers import Dropout
# from keras.applications.resnet50 import preprocess_input, decode_predictions

#input shape
image_input = Input(shape=(224, 224, 3))

#VGG16 
model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

# # model = ResNet50(weights='imagenet')
# model=ResNet50(include_top=False, weights='imagenet', input_tensor=image_input)

model.summary()

last_layer = model.get_layer('fc2').output
# last_layer = model.output
# last_layer = Flatten()(last_layer)
# last_layer = Dropout(0.5)(last_layer)
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

for layer in custom_vgg_model.layers[:-1]:
	layer.trainable = False

custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

t=time.time()

hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=1, validation_data=(X_test, y_test))


print('Training time: %s' % time.time())

(loss, accuracy) = custom_vgg_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

prediction = custom_vgg_model.predict(X_test)
predict = np.argmax(prediction,axis=1)
ans =np.argmax(y_test,axis=1)
print(predict)
print(ans)

265248768/553467096 [=============>................] - ETA: 20:56

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [4]:
data_path = PATH + "/test_images"
data_dir_list = os.listdir(data_path)

test_img_data_list=[]

for i in data_dir_list:
    img = data_path +  '/' + i
    i = image.load_img(img)
    x = image.img_to_array(i)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test_img_data_list.append(x)

test_img_data = np.array(test_img_data_list)

print (test_img_data.shape)  #(600, 1, 224, 224, 3)
test_img_data=np.rollaxis(test_img_data,1,0)
print (test_img_data.shape)  #(1, 600, 224, 224, 3)
test_img_data=test_img_data[0]
print (test_img_data.shape)  #(600, 224, 224, 3)


(10142, 1, 224, 224, 3)
(1, 10142, 224, 224, 3)
(10142, 224, 224, 3)


In [5]:
from tensorflow.keras.models import load_model
custom_vgg_model = load_model('C:/Users/novanto/Downloads/AI AU Materials/FINAL PROJECT 4/custom_vgg_model.h5')
prediction = custom_vgg_model.predict(test_img_data)
predict = np.argmax(prediction,axis=1)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
pd.DataFrame(predict).to_csv("C:/Users/novanto/Downloads/AI AU Materials/FINAL PROJECT 4/predict.csv")

ValueError: Must pass 2-d input